In [12]:
install.packages(c("arules", "recommenderlab", "dplyr", "ggplot2", "ggridges", "tidyr", "coop", "gridExtra", "reshape2"),repos='http://cran.us.r-project.org')

Warning message:
"package 'arules' is not available (for R version 3.6.1)"Warning message:
"dependencies 'arules', 'irlba' are not available"


  There are binary versions available but the source versions are later:
        binary source needs_compilation
dplyr    1.0.6  1.0.7              TRUE
ggplot2  3.3.3  3.3.5             FALSE
tidyr    1.1.3  1.1.4              TRUE
coop     0.6-2  0.6-3              TRUE

  Binaries will be installed
package 'recommenderlab' successfully unpacked and MD5 sums checked
package 'dplyr' successfully unpacked and MD5 sums checked
package 'ggridges' successfully unpacked and MD5 sums checked
package 'tidyr' successfully unpacked and MD5 sums checked
package 'coop' successfully unpacked and MD5 sums checked
package 'gridExtra' successfully unpacked and MD5 sums checked
package 'reshape2' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\vince\AppData\Local\Temp\RtmpMbiQYq\downloaded_packages


installing the source package 'ggplot2'



In [9]:
library(recommenderlab)
library(dplyr)
library(ggplot2)
library(ggridges)
library(tidyr)
library(coop)
library(gridExtra)
library(reshape2)

Warning message:
"package 'recommenderlab' was built under R version 3.6.3"

ERROR: Error: package 'arules' required by 'recommenderlab' could not be found


Aufgabe: Reduziere den MovieLense Datensatz auf rund 400 Kunden 
und 700 Filme, indem du Filme und Kunden  mit sehr wenigen Ratings 
entfernst.
Untersuche und dokumentiere die Eigenschaften des reduzierten 
Datensatzes und beschreibe den Effekt der Datenreduktion:
  1.Anzahl Filme und Kunden sowie Sparsity vor und nach Datenreduktion

In [7]:
data(MovieLense)
MovieLense_df <- as(MovieLense, "data.frame")
MovieLenseMeta_df <- as(MovieLenseMeta, "data.frame")
MOVIE_MIN_RATED = 700
USER_MIN_RATED = 400
SEED = 42
set.seed(SEED)

movies_min_rated <- MovieLense_df %>%
  count(item) %>%
  arrange(desc(n)) %>%
  head(MOVIE_MIN_RATED) %>%
  select(item)

MovieLense_user_movies_reduced_df <- MovieLense_df %>%
  inner_join(movies_min_rated, by='item')

user_min_movies_rated <- MovieLense_user_movies_reduced_df %>%
  count(user) %>%
  arrange(desc(n)) %>%
  head(USER_MIN_RATED) %>%
  select(user)

MovieLense_user_movies_reduced_df <- MovieLense_user_movies_reduced_df %>%
  inner_join(user_min_movies_rated, by='user') 

Movie_Genre <- MovieLenseMeta %>%
  pivot_longer(cols=c(unknown:Western)) %>%
  filter(value==1) %>% 
  rename(genre = name)

Warning message in data(MovieLense):
"data set 'MovieLense' not found"Loading required package: recommenderlab
Warning message:
"package 'recommenderlab' was built under R version 3.6.3"

ERROR: Error in .requirePackage(package): unable to find required package 'recommenderlab'


Aufgabe DIY: Implementiere Funktionen für die Beurteilung der Top-N 
Metriken Precision und Recall sowie für alle Kunden  der Item-space 
Coverage und Novelty und teste diese mit IBCF Recommendations 
(Basis: reduzierter Datensatz; N = 5, 10, 15, 20, 25, 30).


1.Implementiere eine Funktion, um aus Top-N Listen für alle Kunden die Item-
space Coverage@N und Novelty@N eines Recommenders zu beurteilen und 
teste diese.

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)


IBCF_reduced <- Recommender(data = getData(eval_reduced, "train"), method = "IBCF", 
                            parameter = list(k = 3, method = "Cosine", na_as_zero=TRUE))


In [ ]:
top_n_to_df <- function(top_n){
  #' @description Function to transform a TopNList object to a data.frame object.
  #' @param topn Topn List
  topn_df <- data.frame(user = names(as(top_n, 'list')), 
    rating = unlist(top_n@ratings), index = unlist(top_n@items))

  topn_df$item <- top_n@itemLabels[topn_df$index]
  topn_df$year <- MovieLenseMeta$year[topn_df$index]
  topn_df <- topn_df[order(topn_df$user),]
  topn_df
}

calc_coverage <- function(top_n, n_movies=1664){
  #' @description Function, which returns coverage 
  #' @param top_n Top_n list generated by `recommenderlab::predict`
  #' @param n_movies How many total movies there are. In the MovielenseDB, there are 1664 Movies.
  top_n = as(top_n, 'list')
  unique_pred_movies = unique(unlist(top_n, recursive = FALSE))
  n_unique_pred_movies = length(unique_pred_movies)
    
  return (n_unique_pred_movies / n_movies)
}

calc_popularity <- function(MovieLenseData, n_movies=1664){
  #' @description Function, which returns coverage as described here: https://ds-spaces.technik.fhnw.ch/6rsy/2021/05/02/recommender-system-evaluierung-coverage-und-novelty/
  #' @param MovieLenseData Movielense data.frame, containing how a user has rated a movie.
  #' @param n_movies How many total movies there are. In the MovielenseDB, there are 1664 Movies.
  
  popularity_movie <- MovieLenseData %>%
    count(item) %>%
    mutate(popularity = log2(n / n_movies)) %>%
    select(item, popularity)
  return (popularity_movie)
  
}
calc_novelty <- function(top_n, MovieLenseData, n_movies=1664){
  #' @description Function, which returns novelty as described here: https://ds-spaces.technik.fhnw.ch/6rsy/2021/05/02/recommender-system-evaluierung-coverage-und-novelty/
  #' @param top_n Top_n list generated by `recommenderlab::predict`
  #' @param MovieLenseData MovieLense Dataset as data.frame
  #' @param n_movies How many total movies there are. In the MovielenseDB, there are 1664 Movies.
  popularity = calc_popularity(MovieLenseData, n_movies) 
  
  top_n_df <- top_n_to_df(top_n)
    
  group_size_user = top_n_df %>%
    group_by(user) %>%
    summarise(n = n()) %>%
    select(user, n)
  
  top_n_df <- top_n_df %>%
    left_join(group_size_user, by = 'user')
  
  top_n_popularity_df <- top_n_df %>%
    inner_join(popularity, by = 'item') %>%
    mutate(popularity = popularity / n)
  
  novelty = sum(top_n_popularity_df$popularity)
  
  S = length(unique(MovieLenseData$user))
  
  novelty = - novelty / S
  
  return (novelty)
}

In [ ]:
top_n = predict(object = IBCF_reduced, newdata = getData(eval_reduced, "unknown"), n = 15)

In [ ]:
calc_coverage(top_n)
calc_novelty(top_n, MovieLense_df)

In [ ]:
n_list = c(3, 5, 9, 15)
given_list = c(3, 5, 15)
dim_svd = c(5, 20, 30)

In [ ]:
# Test coverage and novelty on 5, 10, 15, 20, 25 and 30 top_n predictions.

coverage <- matrix(, nrow = length(n_list), ncol = length(given_list), dimnames=list(n_list,given_list))
novelty <- matrix(, nrow = length(n_list), ncol = length(given_list), dimnames=list(n_list,given_list))

for(n in 1:length(n_list)){
    for (g in 1:length(given_list)){
    eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                             method="split", train=0.8, k = 3, given=g)
        
    IBCF_reduced <- Recommender(data = getData(eval_reduced, "train"), method = "IBCF", 
                            parameter = list(method = "Cosine", na_as_zero=TRUE, normalize='center'))
    
    top_n = predict(object = IBCF_reduced, newdata = getData(eval_reduced, "unknown"), n = n)
    coverage[n, g] = calc_coverage(top_n)
    novelty[n, g] = calc_novelty(top_n, MovieLense_df)
        }
}


In [ ]:
coverage

Man sieht, dass bei der Coverage der Parameter N einen grossen Einfluss hat. Dagegen hat der Parameter "Given" einen kleineren Einfluss. Wenn man also die Coverage maximieren will, sollte man ein grössmöchglistes N wählen.

In [ ]:
novelty

Die Novelty verändert sich wenig zwischen den Paramater. Dies ergibt aber auch Sinn, da wenn ein Model Grundlegend keine unbekannten Filme empfiehlt, ändert die Top N Liste auch nur wenig dran.

In [ ]:
avg_to_df <- function(evlist, n){
  #' @description gets the confusion matices from `evlist` and transforms it to a single data.frame
  #' @param evlist evaluationResultList
  #' @return data.frame confusion matrix with all recommenders as rec and number of recommendations as n
  evaluation_avg <- data.frame()
  for (i in 1:length(evlist)){
    current_avg <- as.data.frame(avg(evlist[i]))
    colnames(current_avg) <- c('TP','FP','FN','TN','N', 'precision','recall','TPR','FPR', 'n')
    current_avg$specificity <- current_avg$TN * (current_avg$TN + current_avg$FP)**-1
    current_avg$n <- n
    rownames(current_avg) <- NULL
    current_avg$rec <- names(evlist)[i]
    evaluation_avg <- rbind(evaluation_avg, current_avg)
  }
  return(evaluation_avg)
}
plot_percision_recall <- function(evaluation_avg){
  #' @description plots the percision recall curve with the according number of predictions
  #' @param evaluation_avg data.frame from avg_to_df
  ggplot(evaluation_avg, aes(x=precision, y=recall, label=n)) +
    geom_line(aes(color=rec)) +
    geom_text() +
    scale_color_manual(values=1:length(evaluation_avg)) +
    ggtitle('Precision Recall Curve') +
    xlab('precision') +
    ylab('recall')
}


## Implementierung Top-N Monitor
Aufgabe DIY: Untersuche die relative Übereinstimmung zwischen Top-
N Empfehlungen und präferierten Filmen für 4 unterschiedliche 
Modelle (z.B. IBCF und UBCF mit unterschiedlichen  Ähnlichkeits-
metriken / Nachbarschaften sowie SVD mit unterschiedlicher 
Dimensionalitätsreduktion).

### 1.Fixiere 20 zufällig gewählte Testkunden für alle Modellvergleiche

In [ ]:
set.seed(SEED)
random_user_idx = floor(runif(20, min=0, max=nrow(getData(eval_reduced, "unknown"))+1))
test_user_selected = getData(eval_reduced, "unknown")[random_user_idx]

### 2.Bestimme den Anteil der Top-N Empfehlung nach Genres pro Kunde
Zuerst schauen wir, basierend auf Precision, Recall, Novelty und Coverage, welche Modelle überhaupt interessant sind.

In [ ]:
# Random Users:
set.seed(42)

algorithms <- list(
    "IBCF, z-score, cos" = list(name="IBCF", param=list(normalize = "Z-score",
                                                   method = 'cosine')),
    "IBCF, z-score, jac" = list(name="IBCF", param=list(normalize = "Z-score",
                                                   method = 'jaccard')),
    "IBCF, center, cos" = list(name="IBCF", param=list(normalize = "center",
                                                 method = 'cosine')),
    "IBCF, center, jac" = list(name="IBCF", param=list(normalize = "center",
                                                 method = 'jaccard')),
    "UBCF, z-score, cos" = list(name="UBCF", param=list(normalize = "Z-score",
                                                   method = 'cosine')),
    "UBCF, z-score, jac" = list(name="UBCF", param=list(normalize = "Z-score",
                                                   method = 'jaccard')),
    "UBCF, center, cos" = list(name="UBCF", param=list(normalize = "center",
                                                 method = 'cosine')),
    "UBCF, center, jac" = list(name="UBCF", param=list(normalize = "center",
                                                       method = 'jaccard')), 
    "SVD, 2" = list(name='SVD', param=list(k=2)),
    "SVD, 5" = list(name='SVD', param=list(k=5)),
    "SVD, 10" = list(name='SVD', param=list(k=10)),
    "SVD, 20" = list(name='SVD', param=list(k=20))
)


In [ ]:
Experimente mit NN und given und good rating.

In [ ]:
set.seed(SEED)
# Let's check some algorithms against each other
scheme <- recommenderlab::evaluationScheme(coerce(MovieLense_user_movies_reduced_df,MovieLense) , method = "split", train = 0.8, k=3, 
                          given = 3, goodRating = 4)


# run algorithms, predict next n movies
n = c(3, 5, 10, 15, 20, 25, 30)
results <- evaluate(scheme, algorithms, n=n, progress=FALSE)

# Draw ROC curve
plot(results, annotate = c(1,5,9), legend="topleft")

# See precision / recall
plot(results, "prec/rec", annotate=c(1,5,9))

SVD: 

Interessanterweise ist SVD reduziert auf zwei dimensionen am besten.

Precision und Recall ist wie bekannt ein Trade-Off. Man darf das N bei Top N aber nicht zu hoch wählen, da die FPR schneller zunimmt als bei anderen Modellen. SVD ist aber sehr eindrucksvoll, vorallem bei einem kleinen N.


IBCF:

Dieses Modell ist beim Tradeoff mit Precision und Recall stark, im Allgemeinen aber eher schlecht. Ggf gibt es Anwendungen, wenn man Recall reduzieren und Precision maximieren möchte.

UBCF: Verglichen mit SVD hat es für jedes N eine ähnliche FPR und eine niedrigere TPR, es ist also im Schnitt schlechter als SVD

Bei UBCF nimmt die Precision und Recall gleichzeitig zu bei einem höheren N. Im nächsten Schritt schauen wir uns noch die Paramter NN, "given" und "good rating" an von den Modellen mit dem höchsten Precision-Wert. Das wäre also ein eher niedriges N und bei UBCF z-score normalisierung mit Cosine Similarity. 

In [ ]:
# Random Users:
set.seed(42)

algorithms <- list(
    "UBCF, z-score, cos" = list(name="UBCF", param=list(normalize = "Z-score",
                                                   method = 'cosine')),
    "SVD, 2" = list(name='SVD', param=list(k=2))
)


In [ ]:
getResults(results)

In [ ]:
# GOOD_RATING_VALUE
good_rating = 2.5

print(paste("Good Rating Value: ", good_rating))
# Let's check some algorithms against each other
scheme <- recommenderlab::evaluationScheme(coerce(MovieLense_user_movies_reduced_df,MovieLense) , method = "split", train = 0.8, k=3, 
                          given = 3, goodRating = good_rating)


# run algorithms, predict next n movies
n = c(3, 5, 10, 15, 20, 25, 30)
results <- evaluate(scheme, algorithms, n=n, progress = FALSE)

# Draw ROC curve
plot(results, annotate = c(1, 2), legend="topleft", main = paste("Good Rating Value: ", good_rating))

# See precision / recall
#plot(results, "prec/rec", annotate=c(1,2), main = paste("Good Rating Value: ", good_rating))

In [ ]:
# GOOD_RATING_VALUE
good_rating = 3

print(paste("Good Rating Value: ", good_rating))
# Let's check some algorithms against each other
scheme <- recommenderlab::evaluationScheme(coerce(MovieLense_user_movies_reduced_df,MovieLense) , method = "split", train = 0.8, k=3, 
                          given = 3, goodRating = good_rating)


# run algorithms, predict next n movies
n = c(3, 5, 10, 15, 20, 25, 30)
results <- evaluate(scheme, algorithms, n=n, progress = FALSE)

# Draw ROC curve
plot(results, annotate = c(1, 2), legend="topleft", main = paste("Good Rating Value: ", good_rating))

# See precision / recall
#plot(results, "prec/rec", annotate=c(1,2), main = paste("Good Rating Value: ", good_rating))

In [ ]:
# GOOD_RATING_VALUE
good_rating = 4

print(paste("Good Rating Value: ", good_rating))
# Let's check some algorithms against each other
scheme <- recommenderlab::evaluationScheme(coerce(MovieLense_user_movies_reduced_df,MovieLense) , method = "split", train = 0.8, k=3, 
                          given = 3, goodRating = good_rating)


# run algorithms, predict next n movies
n = c(3, 5, 10, 15, 20, 25, 30)
results <- evaluate(scheme, algorithms, n=n, progress = FALSE)

# Draw ROC curve
plot(results, annotate = c(1, 2), legend="topleft", main = paste("Good Rating Value: ", good_rating))

# See precision / recall
#plot(results, "prec/rec", annotate=c(1,2), main = paste("Good Rating Value: ", good_rating))

In [ ]:
top_n_genres <- function(top_n, Movie_Genre, col_name){
    #' @algorithms top_n generated by recommenderlab::predict
    #' @Movie_Genre Movie_Genre melted MovieLenseData, containing at least the genre and the title of a movie
    #' @col_name name for copy of percentages of genres (for easier comparison between different models).
    #' @returns genres of users in the top_n list
    top_n = top_n_to_df(top_n)
    Movie_Genre$item = Movie_Genre$title
    genres_top_n <- top_n %>%
        left_join(Movie_Genre, by='item') %>%
        group_by(user, genre)  %>%
        summarise(n = n()) %>% 
        mutate(percent =  100 *n/sum(n))
    genres_top_n[,col_name] = genres_top_n$percent
    return (genres_top_n)
}

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)


recommender <- Recommender(data = getData(eval_reduced, "train"), method = "IBCF", 
                            parameter = list( method = "Cosine", na_as_zero=TRUE))

top_n = predict(recommender, test_user_selected, n=15)

In [ ]:
IBCF_top_genre = top_n_genres(top_n, Movie_Genre, 'IBCFCos')

In [ ]:
ggplot(IBCF_top_genre, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "IBCF Cosine: Genre of highly rated Movies per User in the Top N list")

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)

recommender <- Recommender(data = getData(eval_reduced, "train"), method = "UBCF", 
                            parameter = list(nn = 25, method = "Cosine"))

top_n = predict(recommender, test_user_selected, n=15)
UBCF_cos_top_genre = top_n_genres(top_n, Movie_Genre, 'UBCFCos')

In [ ]:
ggplot(UBCF_cos_top_genre, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "UBCF Cosine: Genre of highly rated Movies per User in the Top N list")

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)


recommender <- Recommender(data = getData(eval_reduced, "train"), method = "UBCF", 
                            parameter = list(nn = 25, method = "Jaccard"))

top_n = predict(recommender, test_user_selected, n=15)
UBCF_jacc_top_n_genre = top_n_genres(top_n, Movie_Genre, 'UBCFJacc')

In [ ]:
ggplot(UBCF_jacc_top_n_genre, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "UBCF Jaccard: Genre of highly rated Movies per User in the Top N list")

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)


recommender <- Recommender(data = getData(eval_reduced, "train"), method = "SVD", 
                            parameter = list(k = 5))

top_n = predict(recommender, test_user_selected, n=15)
SVD_5_top_n_genre = top_n_genres(top_n, Movie_Genre, 'SVD5')

In [ ]:
ggplot(SVD_5_top_n_genre, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "SVD 5: Genre of highly rated Movies per User in the Top N list")

In [ ]:
set.seed(SEED)
eval_reduced <- recommenderlab::evaluationScheme(data = coerce(MovieLense_user_movies_reduced_df, MovieLense), 
                                                 method="split", train=0.8, given=3)


recommender <- Recommender(data = getData(eval_reduced, "train"), method = "SVD", 
                            parameter = list(k = 10))

top_n = predict(recommender, test_user_selected, n=15)
SVD_10_top_n_genre = top_n_genres(top_n, Movie_Genre, 'SVD10')

In [ ]:
ggplot(SVD_10_top_n_genre, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "SVD 5: Genre of highly rated Movies per User in the Top N list")

3. Bestimme pro Kunde den Anteil nach Genres seiner Top-Filme (=Filme, 
welche vom Kunden die besten Bewertungen erhalten haben),

In [ ]:
fav_genre_of_users <- function(random_user_idx, MovieLenseData, Movie_Genre, top_rating=4){
    #' @random_user_idx vector with random ints
    #' @param MovieLenseData Movielense data.frame, containing how a user has rated a movie.
    #' @param Movie_Genre melted MovieLenseData, containing at least the genre and the title of a movie
    #' @top_rating min rating to consider a movie a top movie for a user (inclusive)
    #' @param returns a data.frame with the genres of top rated movies by the user. 
    test_user_selected = as(names(as(test_user_selected, 'list')), 'matrix') # Ugly code is beautiful too.
    Movie_Genre$item = Movie_Genre$title
    user_top_genres <- MovieLenseData %>%
        filter(user %in% test_user_selected) %>%
        filter(rating >= top_rating) %>%
        left_join(Movie_Genre, by = "item")  %>%
        group_by(user, genre)  %>%
        summarise(n = n()) %>% 
        mutate(percent =  100 *n/sum(n))
    user_top_genres$TopGenres <- user_top_genres$percent
        #select(user, genre, n)
    return (user_top_genres)
}

In [ ]:
fav_genres = fav_genre_of_users(test_user_selected, MovieLense_df, Movie_Genre)

In [ ]:
ggplot(fav_genres, aes(x = user, y = percent, fill = genre)) +
 geom_col() + labs(title = "Genre of highly rated Movies per User")
 #scale_y_continuous(labels = percentage)

4. Vergleiche pro Kunde Top-Empfehlungen und Top-Filmen nach Genres,

In [ ]:
all_top_n_genres <- fav_genres %>%
    full_join(IBCF_top_genre, by=c('user', 'genre')) %>%
    full_join(UBCF_cos_top_genre, by=c('user', 'genre')) %>%
    full_join(UBCF_jacc_top_n_genre, by=c('user', 'genre')) %>%
    full_join(SVD_5_top_n_genre, by=c('user', 'genre')) %>%
    full_join(SVD_10_top_n_genre, by=c('user', 'genre')) %>%
    select_at(vars(-ends_with(".y"))) %>%
    select_at(vars(-ends_with(".x"))) 

all_top_n_genres[is.na(all_top_n_genres)] = 0

all_top_n_genres_melted = melt(all_top_n_genres, id.vars = c("user", "genre"), measure.vars = c("TopGenres", "IBCFCos", "UBCFCos", "UBCFJacc", "SVD5", "SVD10"))

In [ ]:
head(all_top_n_genres_melted)

In [ ]:
all_top_n_genres_melted_u22 <- all_top_n_genres_melted %>%
    filter(user==22)
ggplot(all_top_n_genres_melted_u22, aes(x = variable, y = value, fill = genre)) +
 geom_col() + labs(title = "Genre distribution of User 22")
 #scale_y_continuous(labels = percentage)

In [ ]:
all_top_n_genres_melted_u911 <- all_top_n_genres_melted %>%
    filter(user==911)
ggplot(all_top_n_genres_melted_u22, aes(x = variable, y = value, fill = genre)) +
 geom_col() + labs(title = "Genre distribution of User 911")
 #scale_y_continuous(labels = percentage)

In [ ]:
cols = colnames(all_top_n_genres)[c(4:8)]
df <- data.frame(matrix(ncol = length(cols), nrow = 1))
colnames(df) <- cols

In [ ]:
calc_genre_err <- function(all_top_n_genres, cols){
    #' @all_top_n_genres dataframe containing the top genres from top rated movies of users and predictions done by different models.
    #' @cols Column names with the genre distributions of predictions.
    #' @param returns a data.frame with the absolute error of genre difference between the user rated movies and the predicted movies.
    df <- data.frame(matrix(ncol = length(cols), nrow = 1))
    colnames(df) <- cols
    top_genres <- all_top_n_genres$TopGenres
    for (col in cols){
        df_tmp = sum(abs(top_genres - all_top_n_genres[,col]) / length(top_genres))
        df[1, col] <- df_tmp
    }
    return (df)
}

In [ ]:
genre_error = calc_genre_err(all_top_n_genres, colnames(all_top_n_genres)[c(4:8)])

In [ ]:
melt(genre_error)

In [ ]:
klb(2)

In [ ]:
kulbak kleibler ??

In [ ]:
ggplot(melt(genre_error), aes(x = variable, y = value, fill = variable)) +
 geom_col() + labs(title = "Genre distribution of User 911")
 #scale_y_continuous(labels = percentage)

5.Definiere eine Qualitätsmetrik für Top-N Listen und teste sie.

In [ ]:
ggplot(melt(genre_error), aes(x = variable, y = value, fill = variable)) +
 geom_col() + labs(title = "Genre distribution of User 911")
 #scale_y_continuous(labels = percentage)